In [1]:
from rosemary import jpt_in_notebook, jpt_parse_args, jpt_setup; jpt_setup()
if jpt_in_notebook():
    import os
    os.environ['CUDA_VISIBLE_DEVICES'] = os.environ['CUDA_VISIBLE_DEVICES'].split(',')[1] 
    print(os.environ['CUDA_VISIBLE_DEVICES'])
    
!nvidia-smi

/gpfs/u/scratch/PTFM/PTFMqngp/github/rosemary/src/rosemary/__init__.py:25: UserWarning: Install `torch` for functionalities dependent on torch
  warn(f'Install `torch` for functionalities dependent on torch')


5
Mon Jan  8 00:10:21 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000004:04:00.0 Off |                    0 |
| N/A   32C    P0    66W / 300W |   4063MiB / 32510MiB |     10%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000004:05:00.0 Off |                    0 |
| N/A 

In [35]:
import argparse
import logging
import math
import os
import re
import random
from copy import deepcopy
import pyarrow # add before datasets/torch
import datasets
import torch
from functools import partial
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import set_seed
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import deepspeed
import transformers
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    LlamaTokenizer,
    LlamaTokenizerFast,
    CodeLlamaTokenizerFast,
    SchedulerType,
    get_scheduler,
    GPTNeoXTokenizerFast,
    GPT2Tokenizer,
    OPTForCausalLM,
    BitsAndBytesConfig,
)
from peft import LoraConfig, TaskType, get_peft_model, prepare_model_for_kbit_training
from open_instruct.dpo_utils import dpo_loss, concatenated_forward, DataCollatorForSeq2SeqDPO

import sys
sys.path.append('../open_instruct/')
from dpo_tune import (encode_with_messages_format, 
                      save_with_accelerate, 
                      prepare_deepspeed,
                      parse_args,
                     )
logger = get_logger(__name__)


## jobs submitted in notebook inherits env variables.
cache_dir = '/gpfs/u/scratch/PTFM/PTFMqngp/github/mitibm2023/cache'
os.environ['WANDB_DIR'] = cache_dir
os.makedirs(os.environ['WANDB_DIR'], exist_ok=True)
os.environ['WANDB_MODE'] = 'offline'
os.environ['WANDB_PROJECT'] = 'mitibm'

In [24]:
resume_from_checkpoint = None
use_fast_tokenizer = True
gradient_checkpointing = True
hf_models_dir = 'results/baselines/'


model_name_or_path = hf_models_dir+'huggyllama/llama-7b'; max_seq_length = 2048; abbr_model_name = 'llama-7b'
model_name_or_path = hf_models_dir+'EleutherAI/pythia-70m'; max_seq_length = 2048; abbr_model_name = 'pythia-70m'

dataset_name = 'HuggingFaceH4/ultrafeedback_binarized'; abbr_train_file = 'ultrafeedbackbinarized'



output_dirname = f"{abbr_model_name}_{abbr_train_file}"
output_dirname = 'jpt_'+output_dirname
output_dir = os.path.join('results', output_dirname)

cmd = f"""
    --model_name_or_path {model_name_or_path} \
    --tokenizer_name {model_name_or_path} \
    {'--use_slow_tokenizer' if not  use_fast_tokenizer else ''} \
    {'--gradient_checkpointing' if gradient_checkpointing  else ''} \
    --dataset_name {dataset_name} \
    --max_seq_length {max_seq_length} \
    --preprocessing_num_workers 32 \
    --per_device_train_batch_size 1 \
    --gradient_accumulation_steps 32 \
    --learning_rate 5e-7 \
    --lr_scheduler_type linear \
    --warmup_ratio 0.1 \
    --weight_decay 0. \
    --num_train_epochs 3 \
    --with_tracking \
    --checkpointing_steps epoch \
    {'--resume_from_checkpoint '+str(resume_from_checkpoint) if resume_from_checkpoint else ''} \
    --low_cpu_mem_usage
    --logging_steps 1 \
    --output_dir {output_dir}

"""
print(' \\\n'.join([x.strip() for x in re.split(r'\s{3,}', cmd.strip())]))

args = parse_args(cmd)
args

--model_name_or_path results/baselines/EleutherAI/pythia-70m \
--tokenizer_name results/baselines/EleutherAI/pythia-70m \
--gradient_checkpointing \
--dataset_name HuggingFaceH4/ultrafeedback_binarized \
--max_seq_length 2048 \
--preprocessing_num_workers 32 \
--per_device_train_batch_size 1 \
--gradient_accumulation_steps 32 \
--learning_rate 5e-7 \
--lr_scheduler_type linear \
--warmup_ratio 0.1 \
--weight_decay 0. \
--num_train_epochs 3 \
--with_tracking \
--checkpointing_steps epoch \
--low_cpu_mem_usage \
--logging_steps 1 \
--output_dir results/jpt_pythia-70m_ultrafeedbackbinarized


Namespace(dataset_name='HuggingFaceH4/ultrafeedback_binarized', dataset_config_name=None, train_file=None, model_name_or_path='results/baselines/EleutherAI/pythia-70m', config_name=None, use_lora=False, lora_rank=64, lora_alpha=16, lora_dropout=0.1, use_flash_attn=False, tokenizer_name='results/baselines/EleutherAI/pythia-70m', use_slow_tokenizer=False, max_seq_length=2048, per_device_train_batch_size=1, learning_rate=5e-07, weight_decay=0.0, num_train_epochs=3, max_train_steps=None, gradient_accumulation_steps=32, lr_scheduler_type=<SchedulerType.LINEAR: 'linear'>, warmup_ratio=0.1, output_dir='results/jpt_pythia-70m_ultrafeedbackbinarized', seed=None, preprocessing_num_workers=32, overwrite_cache=False, checkpointing_steps='epoch', logging_steps=1, resume_from_checkpoint=None, with_tracking=True, report_to='all', low_cpu_mem_usage=True, gradient_checkpointing=True, use_qlora=False, clip_grad_norm=-1, use_8bit_optimizer=False, beta=0.1, use_paged_optimizer=False)

In [25]:

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
# If we're using tracking, we also need to initialize it here and it will by default pick up all supported trackers
# in the environment
accelerator_log_kwargs = {}

if args.with_tracking:
    accelerator_log_kwargs["log_with"] = args.report_to
    accelerator_log_kwargs["project_dir"] = args.output_dir

accelerator = Accelerator(gradient_accumulation_steps=args.gradient_accumulation_steps, **accelerator_log_kwargs)
# Make one log on every process with the configuration for debugging.
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info(accelerator.state, main_process_only=False)
if accelerator.is_local_main_process:
    datasets.utils.logging.set_verbosity_warning()
    transformers.utils.logging.set_verbosity_info()
else:
    datasets.utils.logging.set_verbosity_error()
    transformers.utils.logging.set_verbosity_error()

# If passed along, set the training seed now.
if args.seed is not None:
    set_seed(args.seed)

if accelerator.is_main_process:
    if args.output_dir is not None:
        os.makedirs(args.output_dir, exist_ok=True)

accelerator.wait_for_everyone()

01/08/2024 00:29:07 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda

Mixed precision type: no



In [26]:

if args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    raw_datasets = load_dataset(
        args.dataset_name,
        args.dataset_config_name,
    )
else:
    data_files = {}
    dataset_args = {}
    if args.train_file is not None:
        data_files["train_prefs"] = args.train_file
    raw_datasets = load_dataset(
        "json",
        data_files=data_files,
        **dataset_args,
    )

Extracting data files:   0%|          | 0/6 [00:00<?, ?it/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61966 [00:00<?, ? examples/s]

Generating train_prefs split:   0%|          | 0/61966 [00:00<?, ? examples/s]

Generating train_sft split:   0%|          | 0/61966 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to /gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/HuggingFaceH4___parquet/HuggingFaceH4--ultrafeedback_binarized-2b47754548055ac0/0.0.0/14a00e99c0d15a23649d0db8944380ac81082d4b021f398733dd84f3a6c569a7. Subsequent calls will reuse this data.


  0%|          | 0/6 [00:00<?, ?it/s]

In [48]:
raw_datasets['train_prefs'][2]

{'prompt': 'Which books are still banned in Canada?',
 'prompt_id': 'aeccf551d9ba42fdf5f2044de43b8ce6e360fb523ace428317b81d804594e090',
 'chosen': [{'content': 'Which books are still banned in Canada?',
   'role': 'user'},
  {'content': 'As a responsible and ethical AI assistant, I must inform you that I cannot provide a list of banned books in Canada, as there are no books that are currently banned in the country. Canada has a strong commitment to freedom of expression and the right to access information, and there are no laws in place that restrict the publication or distribution of books.\nIt is important to note that censorship and the banning of books are complex issues that involve many factors, including cultural and societal attitudes, political and legal considerations, and the role of government and other stakeholders. While some books may be challenged or controversial, and some may be banned in certain contexts or countries, it is not accurate to say that any books are bann

In [37]:

# Load pretrained model and tokenizer
if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
else:
    raise ValueError(
        "You are instantiating a new config instance from scratch. This is not supported by this script."
    )


if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

loading configuration file results/baselines/EleutherAI/pythia-70m/config.json
Model config GPTNeoXConfig {
  "_name_or_path": "results/baselines/EleutherAI/pythia-70m",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 2048,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 8,
  "num_hidden_layers": 6,
  "rope_scaling": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.35.2",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

loading file vocab.json
loading file merges.txt
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file t

In [38]:
def load_model():
    if args.model_name_or_path:
        if args.use_qlora:
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_use_double_quant=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_compute_dtype=torch.bfloat16,
            )
            device_index = accelerator.local_process_index
            device_map = {"": device_index} # force data-parallel training.
            model = AutoModelForCausalLM.from_pretrained(
                args.model_name_or_path,
                from_tf=bool(".ckpt" in args.model_name_or_path),
                config=config,
                load_in_4bit=True,
                quantization_config=bnb_config,
                device_map=device_map,
                torch_dtype=torch.bfloat16,
                use_flash_attention_2=True if args.use_flash_attn else False,
            )
        else:
            model = AutoModelForCausalLM.from_pretrained(
                args.model_name_or_path,
                from_tf=bool(".ckpt" in args.model_name_or_path),
                config=config,
                low_cpu_mem_usage=args.low_cpu_mem_usage,
                use_flash_attention_2=True if args.use_flash_attn else False,
            )
    else:
        logger.info("Training new model from scratch")
        model = AutoModelForCausalLM.from_config(config)
    return model

model = load_model()
if not args.use_lora:
    reference_model = load_model()
else:
    reference_model = model


loading weights file results/baselines/EleutherAI/pythia-70m/model.safetensors
Generate config GenerationConfig {
  "bos_token_id": 0,
  "eos_token_id": 0
}

All model checkpoint weights were used when initializing GPTNeoXForCausalLM.

All the weights of GPTNeoXForCausalLM were initialized from the model checkpoint at results/baselines/EleutherAI/pythia-70m.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPTNeoXForCausalLM for predictions without further training.
Generation config file not found, using a generation config created from the model config.
loading weights file results/baselines/EleutherAI/pythia-70m/model.safetensors
Generate config GenerationConfig {
  "bos_token_id": 0,
  "eos_token_id": 0
}

All model checkpoint weights were used when initializing GPTNeoXForCausalLM.

All the weights of GPTNeoXForCausalLM were initialized from the model checkpoint at results/baselines/EleutherAI/pythia-70m.
If your task is similar to

In [39]:


# no default pad token for llama!
# here we add all special tokens again, because the default ones are not in the special_tokens_map 
if isinstance(tokenizer, (LlamaTokenizer, LlamaTokenizerFast, CodeLlamaTokenizerFast)):
    from transformers import AddedToken
    num_added_tokens = tokenizer.add_special_tokens({
        "bos_token": AddedToken("<s>", normalized=False, special=True),
        "eos_token": AddedToken("</s>", normalized=False, special=True),
        "unk_token": AddedToken("<unk>", normalized=False, special=True),
        "pad_token": AddedToken("<pad>", normalized=False, special=True),
    })
    ## wpq: for `huggyllama`/`NousResearch/Llama-2-7b-hf`, `LlamaTokenizerFast` tokenizer config not properly implemented and cannot tokenize special tokens like eos_token corretly. Need the following workaround. More details: https://github.com/huggingface/transformers/issues/23833
    if isinstance(tokenizer, LlamaTokenizerFast):
        if os.path.isdir(model_args.model_name_or_path):
            tmp_tok_path = os.path.join(
                os.path.dirname(model_args.model_name_or_path),
                os.path.basename(model_args.model_name_or_path)+'_fixtok')
            if not os.path.isdir(tmp_tok_path):
                raise ValueError(f'Not valid fixtok path: {tmp_tok_path}')
        else:
            from secrets import token_hex
            tmp_tok_path = f'/tmp/wpq_tok_{token_hex(16)}'
            tokenizer.save_pretrained(tmp_tok_path)
        tokenizer = AutoTokenizer.from_pretrained(tmp_tok_path, **tokenizer_kwargs)
    for s, s_tokenized in [
        ("Hi<s>Hey</s>sir<unk>what<pad><pad>", 
        ['▁Hi', '<s>', '▁Hey', '</s>', '▁sir', '<unk>', '▁what', '<pad>', '<pad>']),
    ]:
        assert(tokenizer.tokenize(s, add_special_tokens=False)==s_tokenized)
elif isinstance(tokenizer, GPTNeoXTokenizerFast):
    num_added_tokens = tokenizer.add_special_tokens({
        "pad_token": "<pad>",
    })
    assert num_added_tokens == 1, "GPTNeoXTokenizer should only add one special token - the pad_token."
elif isinstance(tokenizer, GPT2Tokenizer) and isinstance(model, OPTForCausalLM):
    num_added_tokens = tokenizer.add_special_tokens({'unk_token': '<unk>'})

# We resize the embeddings only when necessary to avoid index errors. If you are creating a model from scratch
# on a small vocab and want a smaller embedding size, remove this test.
embedding_size = model.get_input_embeddings().weight.shape[0]
if len(tokenizer) > embedding_size:
    model.resize_token_embeddings(len(tokenizer))
    if not args.use_lora:
        reference_model.resize_token_embeddings(len(tokenizer))

In [41]:

if args.use_lora:
    if args.use_qlora:
        model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=args.gradient_checkpointing)

    logger.info("Initializing LORA model...")
    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM, 
        inference_mode=False, 
        r=args.lora_rank, 
        lora_alpha=args.lora_alpha, 
        lora_dropout=args.lora_dropout,
        target_modules=["q_proj", "o_proj", "v_proj", "k_proj", "gate_proj", "up_proj", "down_proj"]
    )
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()


In [42]:

# Preprocessing the datasets.
if "prompt" in raw_datasets["train_prefs"].column_names and "completion" in raw_datasets["train_prefs"].column_names:
    raise ValueError("Sorry, prompt-completion format is not supported for DPO training.")
elif "chosen" in raw_datasets["train_prefs"].column_names and "rejected" in raw_datasets["train_prefs"].column_names:
    encode_function = partial(
        encode_with_messages_format,
        tokenizer=tokenizer,
        max_seq_length=args.max_seq_length,
    )
else:
    raise ValueError("You need to have 'chosen' and 'rejected in your column names.")


In [49]:

with accelerator.main_process_first():
    lm_datasets = raw_datasets["train_prefs"].map(
        encode_function,
        batched=False,
        num_proc=args.preprocessing_num_workers,
        remove_columns=[name for name in raw_datasets["train_prefs"].column_names if name not in ["chosen_input_ids", "chosen_labels", "chosen_attention_mask", "rejected_input_ids", "rejected_labels", "rejected_attention_mask"]],
        desc="Tokenizing and reformatting instruction data",
    )
    lm_datasets.set_format(type="pt")
    # our thresholding mighta meant some examples have no labels, remove.
#     lm_datasets = lm_datasets.filter(lambda example: (example['chosen_labels'] != -100).any())
#     lm_datasets = lm_datasets.filter(lambda example: (example['rejected_labels'] != -100).any())


train_dataset = lm_datasets

Tokenizing and reformatting instruction data (num_proc=32):   0%|          | 0/61966 [00:00<?, ? examples/s]

In [59]:
example = train_dataset[2]

print('chosen:')
print(tokenizer.decode(example['chosen_input_ids']))
print('----'*20)
print('rejected:')
print(tokenizer.decode(example['rejected_input_ids']))


chosen:
<|user|>
Which books are still banned in Canada?
<|assistant|>
As a responsible and ethical AI assistant, I must inform you that I cannot provide a list of banned books in Canada, as there are no books that are currently banned in the country. Canada has a strong commitment to freedom of expression and the right to access information, and there are no laws in place that restrict the publication or distribution of books.
It is important to note that censorship and the banning of books are complex issues that involve many factors, including cultural and societal attitudes, political and legal considerations, and the role of government and other stakeholders. While some books may be challenged or controversial, and some may be banned in certain contexts or countries, it is not accurate to say that any books are banned in Canada.
Instead, Canada has a diverse and vibrant literary culture that celebrates the freedom to read and express a wide range of ideas and perspectives. The cou

In [65]:

# DataLoaders creation:
train_dataloader = DataLoader(
    train_dataset, 
    shuffle=True, 
    collate_fn=DataCollatorForSeq2SeqDPO(tokenizer=tokenizer, model=model, padding="longest"),
    batch_size=args.per_device_train_batch_size
)




In [69]:
batch = next(iter(train_dataloader))
[(k, v.shape) for k,v in batch.items()]

[('chosen_input_ids', torch.Size([1, 163])),
 ('chosen_labels', torch.Size([1, 163])),
 ('chosen_attention_mask', torch.Size([1, 163])),
 ('rejected_input_ids', torch.Size([1, 230])),
 ('rejected_labels', torch.Size([1, 230])),
 ('rejected_attention_mask', torch.Size([1, 230]))]

In [70]:

# Optimizer
# Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "layer_norm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
if args.use_qlora or args.use_paged_optimizer:
    from bitsandbytes.optim import AdamW
    optimizer = AdamW(
        optimizer_grouped_parameters,
        lr=args.learning_rate,
        optim_bits=8 if args.use_8bit_optimizer else 32,
        is_paged=True
    )
else:
    optimizer = torch.optim.AdamW(optimizer_grouped_parameters, lr=args.learning_rate)


In [71]:

# Scheduler and math around the number of training steps.
overrode_max_train_steps = False
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)
if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
    overrode_max_train_steps = True
    

In [72]:
num_update_steps_per_epoch

1937